In [1]:
import psycopg2
import os
import sys
import requests
import datetime
import time
import re

In [2]:
# db연결
conn = psycopg2.connect(host="ec2-18-207-37-30.compute-1.amazonaws.com", 
                        dbname="da3iiu1dg1eubl", 
                        user="arbmerojlhxbrf", 
                        password="6944d2306202fed548eb3547ca2aaf2cfc420aa21880236efff1ba4f395f35f8", 
                        port="5432")

# 데이터 조작 객체 생성
cur = conn.cursor()

In [3]:
# html 디코딩 함수
def clean_html(x):
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x

In [4]:
#데이터베이스 새로 만들기 (복합 키 세팅)
# 테이블명 : nesdata
# 컬럼명 : nesdate(뉴스작성일), newsname(뉴스타이틀), neslink(뉴스링크)
# create table
def db_create():
    cur.execute("CREATE TABLE IF NOT EXISTS newsdata(newsdate VARCHAR(30) NOT NULL, newsname VARCHAR(100) NOT NULL, newslink VARCHAR(200));")
    conn.commit()

db_create()

In [9]:
# db에 넣기 통합본 - 자동용
def insertnews():

    # db연결
    conn = psycopg2.connect(host="ec2-18-207-37-30.compute-1.amazonaws.com", 
                            dbname="da3iiu1dg1eubl", 
                            user="arbmerojlhxbrf", 
                            password="6944d2306202fed548eb3547ca2aaf2cfc420aa21880236efff1ba4f395f35f8", 
                            port="5432")

    # 데이터 조작 객체 생성
    cur = conn.cursor()

    # 네이버 검색api 키값
    client_id = 'CmmN2OJp47c60h9IWTRs'
    client_secret = 'mALduPDQEB'

    #검색설정
    search_word = '4호선'
    encode_type = 'json'
    max_display = 30
    sort = 'date'
    start = 1

    url = f"https://openapi.naver.com/v1/search/news.{encode_type}?query={search_word}&display={str(int(max_display))}&start={str(int(start))}&sort={sort}"

    #헤더에 아이디와 키 정보 넣기
    headers = {'X-Naver-Client-Id' : client_id,
            'X-Naver-Client-Secret':client_secret
            }

    #HTTP요청 보내기
    r = requests.get(url, headers=headers)
    #요청 결과 보기 200 이면 정상적으로 요청 완료

    # json형식으로 받기
    result = r.json()

    # key : items
    result_items=result['items']

    # db에 저장하기

    for i in result_items:
        
        # 날짜 형식 변환
        pDate = datetime.datetime.strptime(i['pubDate'][5:-6], '%d %b %Y %H:%M:%S')
        pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
        
        # 뉴스제목 디코딩
        title = clean_html(i['title'])
        
        link = i['link']
        
        insertdata = (pDate, title, link)
        
        # ON CONFLICT : 데이터가 중복되면 INSERT하지않음
        cur.execute("INSERT into newsdata(newsdate, newsname, newslink) VALUES (%s, %s, %s) ON CONFLICT (newsname, newslink) DO NOTHING", insertdata)

    # db commit
    conn.commit()   

In [6]:
insertnews()

In [7]:
# select newdata
cur.execute(f"SELECT * FROM newsdata order by newsdate DESC limit 10")
result_all = cur.fetchall()

newslist=""
for i in result_all:
    for j in i:
        newslist = newslist + j + "\n"
    newslist = newslist + "\n"

print(newslist[:-2])

2022-11-22 11:44:00
“서울에 휴양림이 생긴다고?”…노원구, 수락산 자연 휴양림 23일 착공
https://n.news.naver.com/mnews/article/028/0002615914?sid=102

2022-11-22 11:24:00
전장연 이태원 참사 언급 vs 누리꾼 물타기 말라 시끌
https://n.news.naver.com/mnews/article/008/0004820755?sid=102

2022-11-22 11:09:00
작년 서울 지하철 사고·운행장애 16건... 5년 만 최다 기록
https://n.news.naver.com/mnews/article/366/0000856732?sid=102

2022-11-22 10:42:00
노원구, 서울시 최초 수락산 자연 휴양림 조성…23일 착공
http://www.shinailbo.co.kr/news/articleView.html?idxno=1626357

2022-11-22 10:28:00
지하철 타서 늦어요 핑계인 줄 알았는데 알고보니…지하철 사고건수 작년 ...
https://n.news.naver.com/mnews/article/057/0001704431?sid=102

2022-11-22 10:02:00
진접역 오피스텔 투웨니퍼스트 르메트로 공급 예정
http://www.woodkorea.co.kr/news/articleView.html?idxno=71151

2022-11-22 09:44:00
작년 서울 지하철 사고·운행장애 16건...5년만에 최다 기록
https://n.news.naver.com/mnews/article/022/0003756747?sid=102

2022-11-22 09:44:00
전장연 이태원 참사 한 달도 안 됐는데…경찰 충돌 비판
https://n.news.naver.com/mnews/article/015/0004777716?sid=102

2022-11-22 09:36:00
[시선공감] 김창식 도의원 소방서 안전체험관 설치와 운영 활성화 될 것 

In [8]:
conn.commit()